# MapScapingMap

My plan is to create a map with the locations of all the interview partners Danial O'Donohue had on his [MapScaping](https://mapscaping.com/) Podcast.
For this, I scrape his website for the basic info (episode, date, duration, title, categories) on each of the >200 podcast episodes. Then, I have to add the interviewees and their location etc. manually.

### import packages

In [9]:
# import all necessary modules and packages
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import time
warnings.simplefilter(action='ignore', category=FutureWarning)

In [11]:
pd.set_option('display.max_rows', None)  # Optionally, ensure all rows are shown

### scrape website

In [13]:
base_url = 'https://mapscaping.com/podcasts/page/'
page_numbers = range(1, 13)

# Creating a list of URLs for pages 1 to 12 /!\ except page 4, see below!
links_list = [base_url + str(page_number) for page_number in page_numbers if page_number != 4]
# links_list

In [15]:
all_dfs = []

for url in tqdm(links_list): # Iterate over the links and track progress
    request = requests.get(url)
    soup = BeautifulSoup(request.text)

    date_soup = soup.findAll('span', class_= "blog-meta-date-display")
    dates = [date.text for date in date_soup]

    title_soup = soup.findAll('h2', class_= "secondline-blog-title")
    titles = [title.text for title in title_soup]
    titles = [title.replace('\n','').replace('\t','') for title in titles] # not nice, but works

    episode_soup = soup.findAll('div', class_= "blog-meta-serie-episode")
    episodes = [episode.text for episode in episode_soup]

    categories_soup = soup.findAll('span', class_= "blog-meta-category-list")
    categories = [category.text for category in categories_soup]
    categories = [category.replace('\n','').replace('\t','') for category in categories] # not nice, but works

    duration_soup = soup.findAll('span', class_= "blog-meta-time-slt")
    durations = [duration.text for duration in duration_soup]

    current_df = pd.DataFrame({'Episode': episodes, 'Date': dates, 'Title': titles, 'Duration': durations, 'Categories': categories})

    # Append the current DataFrame to the list
    all_dfs.append(current_df)

    # Adding a 5-second delay
    time.sleep(5)

# Concatenate all DataFrames in the list
result_df = pd.concat(all_dfs, ignore_index=True)


100%|████████████████████████████████████████████████████████████████████████████████| 11/11 [01:16<00:00,  6.98s/it]


In [19]:
result_df['Date'] = pd.to_datetime(result_df['Date'])
result_df['Episode'] = result_df['Episode'].str.replace('Episode ', '')
result_df

,Episode,Date,Title,Duration,Categories
0,237,2024-09-25,Female Voices in Geospatial,42:54,mapscaping.com
1,236,2024-09-18,Qfield,49:07,mapscaping.com
2,235,2024-08-28,Analyst To Engineer,41:05,mapscaping.com
3,234,2024-08-16,Satclip – Encoding Location,43:40,mapscaping.com
4,233,2024-08-01,Natural Language Geocoding,45:14,mapscaping.com
5,232,2024-07-10,Semantic Search For Geospatial,50:39,mapscaping.com
6,231,2024-06-05,Why You Should Care About L Band,51:14,mapscaping.com
7,230,2024-05-23,GeoParquet for beginners,42:00,mapscaping.com
8,229,2024-05-16,Finding Stuff Indoors,49:37,mapscaping.com
9,228,2024-05-01,What is humanitarian GIS?,47:24,mapscaping.com


On page 4 of the website:


Episode 162 (QGIS offline and in the field, July 20th, 2022) on page 4 does not have info on duration and episode, which makes it hard to make a dataframe out of it (as the arrays have different length)
In the code below, I manually insert the missing information and bind page 4 to the rest of the scraped data. 

Episode 162 is located:
between 'whitebox tools' and 'sentinel hub'
between 161 and 163

In [23]:
base_url = 'https://mapscaping.com/podcasts/page/4'

response = requests.get(base_url)
soup = BeautifulSoup(response.text)

In [24]:
date_soup = soup.findAll('span', class_= "blog-meta-date-display")
dates = [date.text for date in date_soup]
print(len(dates))
dates

21


['September 28, 2022',
 'September 22, 2022',
 'September 21, 2022',
 'September 14, 2022',
 'September 8, 2022',
 'August 31, 2022',
 'August 18, 2022',
 'August 10, 2022',
 'August 8, 2022',
 'August 3, 2022',
 'July 27, 2022',
 'July 20, 2022',
 'July 20, 2022',
 'July 6, 2022',
 'June 29, 2022',
 'June 25, 2022',
 'June 8, 2022',
 'June 1, 2022',
 'April 28, 2022',
 'April 20, 2022',
 'April 13, 2022']

In [27]:
title_soup = soup.findAll('h2', class_= "secondline-blog-title")
titles = [title.text for title in title_soup]
titles = [title.replace('\n','').replace('\t','') for title in titles] # not nice, but works
print(len(titles))
titles

21


['Thermal Imagery From Space',
 'I Quit My Job',
 'Monitoring Atmospheric Pollution From Space',
 'Hex Tiles',
 'The Business of Web Maps',
 'Finding Water Leaks From Space',
 'How To Keep Your Satellite Pointing At Earth',
 'Bathymetric Lidar and Blue Carbon',
 'Re-Published – QGIS Offline And In The Field',
 'The Open Geospatial Consortium',
 'Monetizing An Open-Source Geospatial Project',
 'Whitebox Tools Is The Backend To Many Frontends',
 'QGIS Offline And In The Field',
 'Sentinel Hub',
 'Unstructured Data Is Dark Data',
 'What Is Modern GIS?',
 'FOSS4G',
 'Building a web based mapping tool into a business',
 'Digital twins – not just a buzzword',
 'Build Your Own SaaS',
 'Getting Your Dream Job in Earth Observation']

In [29]:
episode_soup = soup.findAll('div', class_= "blog-meta-serie-episode")
episodes = [episode.text for episode in episode_soup]
episodes.insert(16, 'Episode 162')
print(len(episodes))
episodes

21


['Episode 174',
 'Episode 173',
 'Episode 172',
 'Episode 171',
 'Episode 170',
 'Episode 169',
 'Episode 168',
 'Episode 167',
 'Episode 166',
 'Episode 165',
 'Episode 164',
 'Episode 161',
 'Episode 163',
 'Episode 160',
 'Episode 159',
 'Episode 158',
 'Episode 162',
 'Episode 157',
 'Episode 156',
 'Episode 155',
 'Episode 154']

In [31]:
categories_soup = soup.findAll('span', class_= "blog-meta-category-list")
categories = [category.text for category in categories_soup]
categories = [category.replace('\n','').replace('\t','') for category in categories] # not nice, but works
print(len(categories))
categories

21


['Earth Observation, Geospatial Career, Geospatial Concepts, Geospatial Startups, Geospatial Tech and Tools',
 'Geospatial Career, GIS',
 'Earth Observation, Geospatial Tech and Tools',
 'Geospatial Concepts, Geospatial Tech and Tools, GIS',
 'Geospatial Startups, Geospatial Tech and Tools, GIS',
 'Earth Observation, Geospatial Startups, Geospatial Tech and Tools',
 'Earth Observation, Geospatial Concepts, Geospatial Tech and Tools',
 'Earth Observation, Geospatial Concepts, Geospatial Tech and Tools',
 'mapscaping.com',
 'Geospatial Career, Geospatial Startups, Geospatial Tech and Tools',
 'Geospatial Startups, Geospatial Tech and Tools, GIS',
 'Geospatial Tech and Tools, GIS',
 'Geospatial Career, Geospatial Tech and Tools, GIS',
 'Earth Observation, Geospatial Concepts, Geospatial Tech and Tools',
 'Geospatial Startups, Geospatial Tech and Tools, GIS',
 'Geospatial Career, Geospatial Tech and Tools, GIS',
 'Geospatial Tech and Tools',
 'Geospatial Career, Geospatial Startups, Geospa

In [33]:
duration_soup = soup.findAll('span', class_= "blog-meta-time-slt")
durations = [duration.text for duration in duration_soup]
durations.insert(16, '35:53')
print(len(durations))
durations

21


['38:51',
 '13:08',
 '38:53',
 '43:06',
 '52:32',
 '39:50',
 '39:55',
 '44:16',
 '35:52',
 '44:56',
 '59:04',
 '50:35',
 '51:06',
 '41:26',
 '48:00',
 '26:10',
 '35:53',
 '49:35',
 '41:54',
 '44:58',
 '53:12']

In [35]:
df_page3 = pd.DataFrame({'Episode': episodes, 'Date': dates, 'Title': titles, 'Duration': durations, 'Categories': categories})
df_page3['Date'] = pd.to_datetime(df_page3['Date'])
df_page3['Episode'] = df_page3['Episode'].str.replace('Episode ', '')

df_page3.info()
df_page3

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Episode     21 non-null     object        
 1   Date        21 non-null     datetime64[ns]
 2   Title       21 non-null     object        
 3   Duration    21 non-null     object        
 4   Categories  21 non-null     object        
dtypes: datetime64[ns](1), object(4)
memory usage: 968.0+ bytes


,Episode,Date,Title,Duration,Categories
0,174,2022-09-28,Thermal Imagery From Space,38:51,"Earth Observation, Geospatial Career, Geospati..."
1,173,2022-09-22,I Quit My Job,13:08,"Geospatial Career, GIS"
2,172,2022-09-21,Monitoring Atmospheric Pollution From Space,38:53,"Earth Observation, Geospatial Tech and Tools"
3,171,2022-09-14,Hex Tiles,43:06,"Geospatial Concepts, Geospatial Tech and Tools..."
4,170,2022-09-08,The Business of Web Maps,52:32,"Geospatial Startups, Geospatial Tech and Tools..."
5,169,2022-08-31,Finding Water Leaks From Space,39:50,"Earth Observation, Geospatial Startups, Geospa..."
6,168,2022-08-18,How To Keep Your Satellite Pointing At Earth,39:55,"Earth Observation, Geospatial Concepts, Geospa..."
7,167,2022-08-10,Bathymetric Lidar and Blue Carbon,44:16,"Earth Observation, Geospatial Concepts, Geospa..."
8,166,2022-08-08,Re-Published – QGIS Offline And In The Field,35:52,mapscaping.com
9,165,2022-08-03,The Open Geospatial Consortium,44:56,"Geospatial Career, Geospatial Startups, Geospa..."


In [37]:
df_complete = pd.concat([result_df, df_page3], ignore_index = True)

In [39]:
final_df = df_complete.sort_values(by='Date')
final_df

,Episode,Date,Title,Duration,Categories
215,1,2019-01-14,The future of collecting and updating geospati...,46:08,"Earth Observation, Geospatial Startups, Geospa..."
214,2,2019-02-08,"Indoor mapping and navigation: Manage, visuali...",40:37,"Geospatial Concepts, Geospatial Tech and Tools..."
213,3,2019-03-10,Bellerby & Co – The globemakers,46:35,GIS
212,4,2019-04-02,freelance mappers create maps for machines,43:43,"Geospatial Startups, Geospatial Tech and Tools..."
211,5,2019-04-08,Powering location intelligence with geo social...,33:48,"Geospatial Concepts, Geospatial Startups, Geos..."
210,6,2019-04-16,Data discovery – the way it should be,38:40,mapscaping.com
209,7,2019-04-25,Augmented reality will change the way you thin...,29:52,"Geospatial Concepts, Geospatial Tech and Tools..."
208,8,2019-05-03,Geo-tagged audio – another way of augmenting r...,29:25,"Geospatial Concepts, Geospatial Startups, Geos..."
207,9,2019-05-08,"Proof of location, bringing the blockchain to ...",26:14,mapscaping.com
206,10,2019-05-15,Mapping Personalised Workplace Risk,32:29,GIS


In [106]:
final_df.to_csv('shiny-map/data/MapScaping_scraped.tsv', sep='\t', index = False)

### add interviewee info

this is the more tricky part. Daniel does not list the names or contact information of his interview partners is a structured way on his website. Therefore, I have to manually go through the transcripts (those he has, luckily!) to find the names of the interview partners and then browse the internet to find social media profiles etc. with data on them

#### Variables I try to find for each interviewee are:
* (interviewee id)
* first name
* last name
* place of residence
* gender (based on pronouns on LinkedIn or how other people/websites refer to them)
* position/seniority *at time of interview*
* weblinks (to LinkedIn/Twitter/personal page and to company page)
* company name *at time of interview*

#### Shortcomings
* not everyone is 'from' somewhere. Several interviewees moved quite often in their lives and don't consider themselves to be from a specific place
* gender is assumed from name & photos if not explicitly stated on LinkedIn or clear from how Daniel refers to them
* the location I use is point data, while some interviewees state a region (or even country) as their location
* quite a few companies that the interviewees worked at the time of the interview were acquired by other firms or were closed
* 

some names can't be written correctly because of UTF8 encoding, e.g.
* Lyden Foust
* Josh Kopecek
* Markus Müller

* several place names (especially PL, CZ) 

Not sure whether the map is too much advertisement for the interview partners / their companies. Daniel does not want the podcast to be a sales pitch for them.

---

I use Excel to type in the data that I found online.
Create custom function to read csv as pandas function causes troubles because of UTF-8 encoing issues

In [102]:
import os

## from 
## https://stackoverflow.com/questions/48812580/python-pandas-unicodedecodeerror-utf-8-codec-cant-decode-byte-0xcd-in-pos

def read_csv(filepath):
     if os.path.splitext(filepath)[1] != '.csv':
          return  # or whatever
     seps = [',', ';', '\t']                    # ',' is default
     encodings = [None, 'utf-8', 'ISO-8859-1']  # None is default
     for sep in seps:
         for encoding in encodings:
              try:
                  return pd.read_csv(filepath, encoding=encoding, sep=sep)
              except Exception:  # should really be more specific 
                  pass
     raise ValueError("{!r} is has no encoding in {} or seperator in {}"
                      .format(filepath, encodings, seps))

In [104]:
df = read_csv("shiny-map/data/MapScaping_extended.csv")

In [65]:
# double square brackets to keep it as DataFrame (instead of Series)
locations = df[['location']]
# drop duplicate locations to make geocoding faster and avoid merge complications later on
locations = locations.drop_duplicates(subset='location')

locations #.info()

,location
0,San Francisco Bay Area
1,NaN
2,"London, England, Islington, Borough of Hackney..."
4,"Cincinnati, Ohio, USA"
5,"Auckland, Auckland, New Zealand"
6,"Toronto, Ontario, Canada"
7,"Okehampton, England, United Kingdom"
8,"San Diego, California, USA"
9,"Austin, Texas, USA"
10,"Heilbad Heiligenstadt, Thuringia, Germany"


### geocode addresses

use Nomatim API for this

In [67]:
import pandas as pd
import requests
import time

# Load addresses from CSV
addresses_df = locations # pd.read_csv('location.csv')
addresses = addresses_df['location'].tolist()

# Function to geocode address using Nominatim API
def geocode_address(address):
    url = 'https://nominatim.openstreetmap.org/search'
    headers = {'User-Agent': 'Nicolas'}
    params = {'q': address, 'format': 'json'}
    
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        results = response.json()
        if results:
            return results[0]['lat'], results[0]['lon']  # Return the latitude and longitude of the first result
    return None, None  # Return None if no results or an error occurred

# Create a list to hold geocoded results
geocoded_addresses = []

# Geocode each address
for address in tqdm(addresses):
    lat, lon = geocode_address(address)
    geocoded_addresses.append({'address': address, 'latitude': lat, 'longitude': lon})
    time.sleep(1)  # Sleep to respect Nominatim's usage policy

# Convert results to a DataFrame
geocoded_df = pd.DataFrame(geocoded_addresses)

# Optionally, save the geocoded results to a new CSV file
geocoded_df.to_csv('shiny-map/data/geocoded_addresses.csv', index=False)

print('Geocoding complete. Results saved to geocoded_addresses.csv.')


100%|██████████████████████████████████████████████████████████████████████████████| 144/144 [04:04<00:00,  1.70s/it]

Geocoding complete. Results saved to geocoded_addresses.csv.


In [69]:
geocoded_df
# manually insert coordinates for...
# ...Bellerby & Co., as it's too specific for nomatim 
geocoded_df.at[2, 'latitude'] = 51.5625709
geocoded_df.at[2, 'longitude'] = -0.0788484
# ...Cologne-Bonn-Region in Germany, as it's unknown to nomatim 
geocoded_df.at[112, 'latitude'] = 50.8285133
geocoded_df.at[112, 'longitude'] = 6.9960294
# ...NaN, as Nominatims geocoding obviously makes no sense at all :D
geocoded_df.at[1, 'latitude'] = 'NaN'
geocoded_df.at[1, 'longitude'] = 'NaN'

geocoded_df

,address,latitude,longitude
0,San Francisco Bay Area,37.7884969,-122.3558473
1,NaN,NaN,NaN
2,"London, England, Islington, Borough of Hackney...",51.562571,-0.078848
3,"Cincinnati, Ohio, USA",39.1014537,-84.5124602
4,"Auckland, Auckland, New Zealand",-36.852095,174.7631803
5,"Toronto, Ontario, Canada",43.6534817,-79.3839347
6,"Okehampton, England, United Kingdom",50.7376667,-4.000316
7,"San Diego, California, USA",32.7174202,-117.162772
8,"Austin, Texas, USA",30.2711286,-97.7436995
9,"Heilbad Heiligenstadt, Thuringia, Germany",51.3756186,10.138224


In [71]:
# drop rows without location / coordinates
# create explicit copy of the DataFrame
geocoded_df_clean = geocoded_df.dropna().copy()
geocoded_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143 entries, 0 to 143
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   address    143 non-null    object
 1   latitude   143 non-null    object
 2   longitude  143 non-null    object
dtypes: object(3)
memory usage: 4.5+ KB


In [73]:
# transform into float and round to 7 decimal points (approx 1cm)
geocoded_df_clean.loc[:, 'latitude'] = geocoded_df_clean['latitude'].astype(float).round(7)
geocoded_df_clean.loc[:, 'longitude'] = geocoded_df_clean['longitude'].astype(float).round(7)

In [75]:
geocoded_df_clean = geocoded_df_clean.rename(columns={'address': 'location'})
geocoded_df_clean.info()
df#.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143 entries, 0 to 143
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   location   143 non-null    object 
 1   latitude   143 non-null    float64
 2   longitude  143 non-null    float64
dtypes: float64(2), object(1)
memory usage: 4.5+ KB


,Episode,Date,Title,Duration,Categories,interviewee_id,first_name,last_name,gender,position,interviewee_link,company_name,location,page_on_website,company_link,incomplete,multiple_guest
0,1,14.01.19,The future of collecting and updating geospati...,46:08:00,"Earth Observation, Geospatial Startups, Geospa...",1.0,Ariel,Seidman,m,"CEO, Founder",https://www.linkedin.com/in/aseidman/,Hivemapper,San Francisco Bay Area,11,https://hivemapper.com/,0,1
1,2,08.02.19,"Indoor mapping and navigation: Manage, visuali...",40:37:00,"Geospatial Concepts, Geospatial Tech and Tools...",2.0,Chris,Wiegand,m,"CEO, Founder",https://www.linkedin.com/in/chris-wiegand-4790...,Jibestream,NaN,11,https://apps.jibestream.com/,1,0
2,3,10.03.19,Bellerby & Co Ð The globemakers,46:35:00,GIS,3.0,Peter,Bellerby,m,"CEO, Founder",https://bellerbyandco.com/meet-the-team/,Bellerby & Co,"London, England, Islington, Borough of Hackney...",11,https://bellerbyandco.com/,0,0
3,4,02.04.19,freelance mappers create maps for machines,43:43:00,"Geospatial Startups, Geospatial Tech and Tools...",4.0,Nikhil,Naikal,m,CEO,https://www.linkedin.com/in/nnaikal/,Mapper.ai,San Francisco Bay Area,11,https://mapper.ai/about/,0,0
4,5,08.04.19,Powering location intelligence with geo social...,33:48:00,"Geospatial Concepts, Geospatial Startups, Geos...",5.0,LØden,Foust,m,"CEO, Founder",https://www.linkedin.com/in/lydenfoust/,Spatial.ai,"Cincinnati, Ohio, USA",11,https://www.spatial.ai/,0,0
5,6,16.04.19,Data discovery Ð the way it should be,38:40:00,NaN,6.0,Anne,Harper,f,Business Development Manager,https://www.linkedin.com/in/anne-harper-65663b...,Koordinates,"Auckland, Auckland, New Zealand",11,https://koordinates.com/,0,0
6,7,25.04.19,Augmented reality will change the way you thin...,29:52:00,"Geospatial Concepts, Geospatial Tech and Tools...",7.0,Alec,Pestov,m,"CEO, Founder",https://www.linkedin.com/in/alec-pestov/?origi...,vGIS,"Toronto, Ontario, Canada",11,https://www.vgis.io/,0,0
7,8,03.05.19,Geo-tagged audio Ð another way of augmenting r...,29:25:00,"Geospatial Concepts, Geospatial Startups, Geos...",8.0,Josh,Kope_ek,m,"Director, Founder",https://www.linkedin.com/in/joshkopecek/,Echoes.xyz,"Okehampton, England, United Kingdom",11,https://echoes.xyz/,0,0
8,9,08.05.19,"Proof of location, bringing the blockchain to ...",26:14:00,NaN,9.0,Arie,Trouw,m,"CEO, CTO, Founder",https://www.linkedin.com/in/arietrouw/,XYO,"San Diego, California, USA",11,https://xyo.network/,0,0
9,10,15.05.19,Mapping Personalised Workplace Risk,32:29:00,GIS,10.0,Clint,van Marrewijk,m,"Director, Founder",https://www.linkedin.com/in/clint-van-marrewij...,SaferMe,"Austin, Texas, USA",11,https://www.safer.me/features/,0,0


In [77]:
# merge geocoded coordinates to main dataframe
result_df = pd.merge(df, geocoded_df_clean, on='location', how='left')

### feature engineering

In [79]:
result_df['Date'] = pd.to_datetime(result_df['Date'], format='%d.%m.%y')
# check that datetime conversion worked properly
result_df = result_df.sort_values(by=['Date'])
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 238 entries, 0 to 237
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Episode           238 non-null    int64         
 1   Date              238 non-null    datetime64[ns]
 2   Title             238 non-null    object        
 3   Duration          238 non-null    object        
 4   Categories        201 non-null    object        
 5   interviewee_id    226 non-null    float64       
 6   first_name        237 non-null    object        
 7   last_name         236 non-null    object        
 8   gender            237 non-null    object        
 9   position          235 non-null    object        
 10  interviewee_link  236 non-null    object        
 11  company_name      235 non-null    object        
 12  location          235 non-null    object        
 13  page_on_website   238 non-null    int64         
 14  company_link      236 non-

In [83]:
# Creating a combined name column for convenience
result_df['full_name'] = result_df['first_name'] + ' ' + result_df['last_name']

# Counting occurrences and creating a new column
result_df['number_of_interviews'] = result_df.groupby('full_name')['full_name'].transform('count') #.astype(int)

In [85]:
# who appears how often?
result_df.sort_values(by=['number_of_interviews', 'full_name'], ascending = False)

,Episode,Date,Title,Duration,Categories,interviewee_id,first_name,last_name,gender,position,...,company_name,location,page_on_website,company_link,incomplete,multiple_guest,latitude,longitude,full_name,number_of_interviews
18,19,2019-07-17,A geospatial story and some housekeeping,38:05:00,NaN,19.0,Daniel,O'Donohue,m,"CEO, Founder",...,MapScaping,"Vejle, Syddanmark, Denmark",10,https://mapscaping.com/,0,1,55.706692,9.536565,Daniel O'Donohue,8.0
40,41,2019-12-26,Trends in geospatial and initiatives in the co...,15:02,NaN,19.0,Daniel,O'Donohue,m,"CEO, Founder",...,MapScaping,"Vejle, Syddanmark, Denmark",0,https://mapscaping.com/,0,1,55.706692,9.536565,Daniel O'Donohue,8.0
86,87,2020-12-03,"My Story, my why",19:19,Geospatial Career,19.0,Daniel,O'Donohue,m,"CEO, Founder",...,MapScaping,"Vejle, Syddanmark, Denmark",0,https://mapscaping.com/,0,1,55.706692,9.536565,Daniel O'Donohue,8.0
126,127,2021-09-22,Geospatial Side Hustles,38:59:00,"Geospatial Career, Geospatial Startups",19.0,Daniel,O'Donohue,m,"CEO, Founder",...,MapScaping,"Vejle, Syddanmark, Denmark",0,https://mapscaping.com/,0,1,55.706692,9.536565,Daniel O'Donohue,8.0
139,140,2021-12-22,Voice Message,02:30,NaN,19.0,Daniel,O'Donohue,m,"CEO, Founder",...,MapScaping,"Vejle, Syddanmark, Denmark",0,https://mapscaping.com/,0,1,55.706692,9.536565,Daniel O'Donohue,8.0
140,141,2022-01-05,What Is Your Idea?,17:33,"Geospatial Career, Geospatial Startups",19.0,Daniel,O'Donohue,m,"CEO, Founder",...,MapScaping,"Vejle, Syddanmark, Denmark",0,https://mapscaping.com/,0,1,55.706692,9.536565,Daniel O'Donohue,8.0
146,147,2022-02-16,Business Ideas For Geospatial People,30:27:00,"Earth Observation, Geospatial Career, Geospati...",19.0,Daniel,O'Donohue,m,"CEO, Founder",...,MapScaping,"Vejle, Syddanmark, Denmark",0,https://mapscaping.com/,0,1,55.706692,9.536565,Daniel O'Donohue,8.0
172,173,2022-09-22,I Quit My Job,13:08,"Geospatial Career, GIS",19.0,Daniel,O'Donohue,m,"CEO, Founder",...,MapScaping,"Vejle, Syddanmark, Denmark",0,https://mapscaping.com/,0,0,55.706692,9.536565,Daniel O'Donohue,8.0
52,53,2020-04-01,Dynamic vector tiles straight from the database,36:02:00,"Geospatial Concepts, Geospatial Tech and Tools...",49.0,Paul,Ramsey,m,Executive Geospatial Engineer,...,Crunchy Data,"Victoria, British Columbia, Canada",9,https://www.crunchydata.com/,0,1,48.428318,-123.364953,Paul Ramsey,5.0
76,77,2020-09-16,Spatial SQL Ð GIS without the GIS,41:10:00,"Geospatial Concepts, Geospatial Tech and Tools...",49.0,Paul,Ramsey,m,Executive Geospatial Engineer,...,Crunchy Data,"Victoria, British Columbia, Canada",7,https://www.crunchydata.com/,0,1,48.428318,-123.364953,Paul Ramsey,5.0


In [87]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 238 entries, 0 to 237
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Episode               238 non-null    int64         
 1   Date                  238 non-null    datetime64[ns]
 2   Title                 238 non-null    object        
 3   Duration              238 non-null    object        
 4   Categories            201 non-null    object        
 5   interviewee_id        226 non-null    float64       
 6   first_name            237 non-null    object        
 7   last_name             236 non-null    object        
 8   gender                237 non-null    object        
 9   position              235 non-null    object        
 10  interviewee_link      236 non-null    object        
 11  company_name          235 non-null    object        
 12  location              235 non-null    object        
 13  page_on_website     

In [89]:
# subset only columns that are relevant
subset = result_df[['Episode', 'Date', 'Title', 'Duration', 'Categories', 'first_name', 'last_name', 'interviewee_link', 'gender', 'position', 'company_name', 'company_link', 'location', 'latitude', 'longitude', 'full_name', 'number_of_interviews']]

In [91]:
subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 238 entries, 0 to 237
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Episode               238 non-null    int64         
 1   Date                  238 non-null    datetime64[ns]
 2   Title                 238 non-null    object        
 3   Duration              238 non-null    object        
 4   Categories            201 non-null    object        
 5   first_name            237 non-null    object        
 6   last_name             236 non-null    object        
 7   interviewee_link      236 non-null    object        
 8   gender                237 non-null    object        
 9   position              235 non-null    object        
 10  company_name          235 non-null    object        
 11  company_link          236 non-null    object        
 12  location              235 non-null    object        
 13  latitude            

In [108]:
# save as tab-separated file so that commas in addresses don't mess it up
subset.to_csv('shiny-map/data/geocoded_addresses.tsv', sep='\t', index = False)

### future tasks

### To Do Preprocessing
* rename variables to be meaningful
* create variable with count of interviews per person
* change data types to time / timedelta? Or only later in postgis??





### To Do in Angular Frontend
* jitter points so that they are not exactly on top of each other
* clustering of points that overlap too much (solve css problems)
    * use colorcoding for percentage of male/female interviewees in cluster
* handle special characters correctly (in names)
* include more in popup
    * interviewee name (make link clickable)
    * episode
    * company name (make link clickable)
* allow filtering for 
    * category (use regex to check whether category is in string)
    * gender
    * date (range filter?)
    * number of interviews (checkbox)
    * interviewee name?
    * company name?

To introduce a jitter of approximately 1 kilometer to the locations in your interviewees table to ensure that identical locations don’t overlay each other, you can use the ST_Translate function in PostGIS. This function shifts the geometry by a specified amount in the x (longitude) and y (latitude) directions. The amount to shift can be determined based on the degree equivalent of the desired distance at the given latitude (keeping in mind that a degree of longitude varies in actual distance depending on the latitude, but for small distances like 1km, this can be approximated fairly accurately).

For the jitter, we can use random offsets in both the latitude and longitude directions. Given that 1 degree of latitude is approximately 111 kilometers, a shift of about 0.009° in any direction would approximate to 1 kilometer. For longitude, this value needs to be adjusted based on the latitude due to the varying distance covered by a degree of longitude, but for simplicity, we'll use the same approximation, which is reasonable for small distances and near the equator.

This SQL statement will move each point in a random direction by up to approximately 1 kilometer. The RANDOM() function generates a value between 0 and 1, so RANDOM() * 0.018 - 0.009 will produce a shift ranging from -0.009° to +0.009° in both latitude and longitude, effectively creating a jitter around the original point.

not needed
* interviewee_id
* page_on_website
* incomplete

**Attention!!!** 
* there are two different Eric Jensen's
* Peter Petrik's episode was re-published because of audio-quality